In [1]:
from cleaner import TextCleanerTransformer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

from nltk.tokenize import WordPunctTokenizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

import pandas as pd
import numpy as np

import re

/home/echo66/.local/share/virtualenvs/ldssa-nlp-Yrltft6c/lib64/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/echo66/.local/share/virtualenvs/ldssa-nlp-Yrltft6c/lib64/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
all_train = pd.read_csv('data/train.csv')

all_train.dropna(inplace=True)

le = LabelEncoder()

le.fit(all_train['Label'].values)

all_train.Label = le.transform(all_train['Label'].values)

train, test = train_test_split(all_train, test_size=0.3, random_state=10, shuffle=True)

In [3]:
text_clf = Pipeline([
    ('stemm', TextCleanerTransformer(
        WordPunctTokenizer(), 
        SnowballStemmer("portuguese", ignore_stopwords=True), 
        [])),
    ('tfidf', TfidfVectorizer(strip_accents='unicode', lowercase=True)), 
    ('clf', LinearSVC(random_state=10))
])

#text_clf.fit(map(str, train['Request'].values), train['Label'].values)

In [4]:
fit_params = {
    'tfidf__norm': [None, 'l1', 'l2'], 
    #'tfidf__use_idf': [False, True], 
    #'tfidf__smooth_idf': [False, True], 
    #'tfidf__sublinear_tf': [False, True], 
    'tfidf__ngram_range': [
        (1, 1), 
        (1, 2), 
        (1, 3), 
        (2, 2), 
        (2, 3)
    ], 
    'tfidf__max_features': [50, 100, 1000, 10000, 50000], 
    'clf__C': [1e-3, 1e-2, 1e-1, 1, 10], 
    'clf__penalty': ['l2']
}

In [5]:
gcv = GridSearchCV(text_clf, fit_params, 
                   scoring='accuracy', 
                   cv=3, verbose=True, 
                   n_jobs=-1)

In [6]:
gcv.fit(train['Request'], train['Label'].values)

Fitting 3 folds for each of 375 candidates, totalling 1125 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed: 13.0min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed: 24.2min
[Parallel(n_jobs=-1)]: Done 1125 out of 1125 | elapsed: 36.3min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('stemm', TextCleanerTransformer(lower=True, regex_list=[], remove_punct=True,
            stemmer=<nltk.stem.snowball.SnowballStemmer object at 0x7f4a51265a90>,
            tokenizer=WordPunctTokenizer(pattern='\\w+|[^\\w\\s]+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL... max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=10, tol=0.0001,
     verbose=0))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'tfidf__norm': [None, 'l1', 'l2'], 'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3), (2, 2), (2, 3)], 'tfidf__max_features': [50, 100, 1000, 10000, 50000], 'clf__C': [0.001, 0.01, 0.1, 1, 10], 'clf__penalty': ['l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=True)

In [8]:
#y_proba = gcv.best_estimator_.predict_proba(map(str, test['Request'].values))

y_pred = gcv.best_estimator_.predict(map(str, test['Request'].values))

from sklearn.metrics import matthews_corrcoef

In [9]:
real_test = pd.read_csv('data/test.csv')

real_test_y_pred = gcv.best_estimator_.predict(map(str, real_test['Request'].values))

real_test['Label'] = le.inverse_transform(real_test_y_pred)

real_test.to_csv('submissions/model_2.csv', index=False)

/home/echo66/.local/share/virtualenvs/ldssa-nlp-Yrltft6c/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [12]:
gcv.best_estimator_.get_params()

{'memory': None,
 'steps': [('stemm',
   TextCleanerTransformer(lower=True, regex_list=[], remove_punct=True,
               stemmer=<nltk.stem.snowball.SnowballStemmer object at 0x7f4a5124ffd0>,
               tokenizer=WordPunctTokenizer(pattern='\\w+|[^\\w\\s]+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>))),
  ('tfidf',
   TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=50000, min_df=1,
           ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
           stop_words=None, strip_accents='unicode', sublinear_tf=False,
           token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
           vocabulary=None)),
  ('clf', LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
        intercept_scaling=1, loss='squared_hinge', max_iter=1000,
        multi_class='ov